## Description:
1. Deeper clean on the data in prepration for analysis/modeling
2. Categorization of string value data

### Header

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Audio
import pandas as pd
import numpy as np
import re
import pickle
import os
import ast
import json
from pandas.api.types import CategoricalDtype
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

## Main

In [39]:
data_dict_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/Test_Data_Dict/'
#saved_data_html_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/HTML/'
os.chdir(data_dict_path)

csv_data_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/CSV_Data/'
file_name = 'CLEANED_Aggregated_Home_Data.csv'
#pd.DataFrame.to_csv(data, path_or_buf= csv_data_path + file_name)
#data = get_aggregatedData(os.listdir(data_dict_path)[:-1])
data = pd.read_csv(csv_data_path+file_name)

data = data.set_index('House_Id')
data = data.drop(axis=1, columns=[
    'Full_Bathrooms', 'Price_Cut_Date', 'Parcel_Number',
    'Has_Hoa_Fee', 'Foundation', 'Has_Cooling', 'Parking', 'Type', 'Heating_Features'
])
# Remove Cooling & Foundation

#data = data.drop(axis=1, columns=['Year_Built:', 'Living_Area', 'Lot:', 'Rent_Control'])
data.Home_Type.loc[(data.Bathrooms == 0) & (data.Bedrooms == 0) & (data.Year_Built.isna()) & (data.Garage_Spaces ==0) & (data.Home_Type.isna())] = 'VACANT_LOT'
data.Home_Type.loc[data.Home_Type == 'MANUFACTURED'] = 'SINGLE_FAMILY'

#Remove Vacant Lots and Foreclosed Homes with unknown prices
data = data[~data.Price_Unknown_Binary]
data = data[~data.Home_Type.isin(['VACANT_LOT'])]

#Create home type categories
home_type_cat_values = ['NO DATA', 'TOWNHOUSE', 'CONDO', 'SINGLE_FAMILY', 'MULTI_FAMILY','APARTMENT']
home_type_cat_type = CategoricalDtype(categories=home_type_cat_values, ordered=True)
data.Home_Type = data.Home_Type.fillna('NO DATA').astype(home_type_cat_type)

#Create nearest store categories
data.Store_Name = data.Store_Name.fillna('NO DATA')
store_name_cat_list = data.Store_Name.value_counts()[data.Store_Name.value_counts().values >= 5].index
other_cat = data.Store_Name[~data.Store_Name.isin(store_name_cat_list)]
data.Store_Name[other_cat.index] = 'Other'
data.Store_Name = data.Store_Name.astype('category')

# Remove 0 from School Distance and School Rating: replace with Median Values after Outliers
data.Average_School_Rating.replace(0, np.nan, inplace=True)
data.Average_School_Rating = replace_nan_median(data.Average_School_Rating, remove_outliers_binary=True)
data.Average_School_Distance.replace(0, np.nan, inplace=True)
data.Average_School_Distance = replace_nan_median(data.Average_School_Distance, remove_outliers_binary=True)

#Filling Store Price level with median value
data.Store_Price_Level = data.Store_Price_Level.fillna(data.Store_Price_Level.median())

#Filling Year Built with Median value
data.Year_Built = data.Year_Built.fillna(data.Year_Built.median())

#Filling Calculated SQFT with 0
data.Calculated_Price_Sqft = data.Calculated_Price_Sqft.fillna(0)

#Filling in Tax Assessed Value Amount based on the Outlier removal median Price/Tax Assessed Value ratio
tav_to_price = data.Tax_Assessed_Value/data.Price
tav_to_price_median = tav_to_price[np.abs(tav_to_price-tav_to_price.mean()) <= (3*tav_to_price.std())].median()
TAV_NAN = data.Tax_Assessed_Value[data.Tax_Assessed_Value.isna()]
TAV_NAN_Prices = data.Price[TAV_NAN.index]
data.Tax_Assessed_Value.loc[TAV_NAN.index] = TAV_NAN_Prices * tav_to_price_median

#Filling in Annual Tax Amount based on the Outlier removal median Price/Tax Amount Value ratio
tax_to_price = data.Annual_Tax_Amount / data.Price
tax_to_price_median = tax_to_price[np.abs(tax_to_price-tax_to_price.mean()) <= (3*tax_to_price.std())].median()
tax_NAN = data.Annual_Tax_Amount[data.Annual_Tax_Amount.isna()]
tax_NAN_Prices = data.Price[tax_NAN.index]
data.Annual_Tax_Amount.loc[tax_NAN.index] = tax_NAN_Prices * tax_to_price_median

# Add Categorical Columns
cat_data = string_to_categories(data, selected_cat_cols, category_mapping_dict)
data = data.merge(cat_data, left_index=True, right_index=True)
data = data.drop(selected_cat_cols, axis=1)

#Fill na with 0 for lot size; tried to fill with median value based on living area to lot size ratio
data.Lot_Size = data.Lot_Size.fillna(data.Lot_Size.median())

# Gets all of the Zipcodes for each ZPID
zpid_zip_code = get_zipcode(data.Address)
data = data.merge(zpid_zip_code, left_index=True, right_index=True)

data.head().T

C:\Users\austi\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\austi\Anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


House_Id,2096667469_zpid,2090771520_zpid,2085043646_zpid,88380061_zpid,2083788163_zpid
Bedrooms,2.00,2.00,1.00,2.00,3.00
Total_Interior_Livable_Area,1182.00,1250.00,840.00,2300.00,2450.00
Number_Of_Appliances,8,8,0,0,4
Bathrooms,2.00,2.00,1.00,3.00,4.00
Stories,1.00,1.00,0.00,0.00,0.00
Home_Type,TOWNHOUSE,TOWNHOUSE,CONDO,CONDO,TOWNHOUSE
Year_Built,1905.00,1905.00,1920.00,2002.00,2019.00
New_Construction,0.00,0.00,0.00,0.00,0.00
Hoa_Fee,254,254,292,0,461
Average_School_Rating,3.50,3.50,3.50,3.50,3.50


In [40]:
#file_name = 'FULLY_CLEANED_Aggregated_Home_Data_8.27.2019.csv'
#pd.DataFrame.to_csv(data, path_or_buf= csv_data_path + file_name)
pkl_dataframes_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/Pickle_DataFrames/'
pkl_file_name = 'Fully_Cleaned_House_Data_8.27.2019.pkl'
data.to_pickle(path=pkl_dataframes_path + pkl_file_name)

### Notes

###### Notes
1. Appliances: get the number of appliances included in sale, not the listing
2. Get the binary value of if price cut or not
3. Get Percentage away from listing from zestimate
4. Have Dataframe and do a merge from the main dataframe with the created data frames from the dict
    * Make a separate dictionary for each
    * If not listed initially, go forward and look for these:
        1. Zestimate
        2. Parking
        3. Heating / Cooling
        4. Home Type
5. Loop through Zip Code PKL files and find the top 50% - 75% of Dictionary Keys listed
    * Gather the top 50% - 75% and find the top values across the board again
6. Gather the Descriptions for each home and match them for Sentiment Analysis
    * Base the sentiment analysis on the price of the home or price per square foot, or price/Lot Size

###### Data Cleaning Notes
1. Try to mark empty lot based on:
    * absent bed/bath and year build
    * small number of photos: below the average
    * Home_Type
    * Large Living Area or Lot Size
    * Beds/Baths = 0; Home_Type == NaN; Heating_Features == NaN; Number_Of_Appliances == 0;
2. Categorize These Columns:
    * Exterior Features
    * Parking Features -> Parking
    * Home_Type
    * Foundation
    * Construction_Materials
    * Architectural_Style
    * Heating
    * Cooling
    * Structure Type
3. Numerical Changes:
    * Hoa Fee
    * Tax_Assessed_Value
    * Annual_Tax_Amount

## Data Structures

In [2]:
category_mapping_dict = {
    'Exterior_Features':{
        'BRICK':['BRICK', 'CONCRETE', 'CEMENT' 'MASONRY', 'STONE'],
        'WOOD': ['WOOD', 'CEDAR', 'SHINGLE'],
        'STUCCO': ['STUCCO'],
        'VINYL': ['VINYL'],
        'METAL': ['ALUMINUM', 'METAL']
    },
    'Construction_Materials':{
        'BRICK': ['BRICK', 'CONCRETE', 'CEMENT', 'MASONRY', 'STONE'],
        'WOOD': ['WOOD', 'CEDAR'],
        'VINYL': ['VINYL'],
        'METAL': ['ALUMINUM', 'METAL']
    },
    'Basement':{
        'FINISHED': ['FINISHED'],
        'PARTIALLY FINISHED': ['PARTIALLY FINISHED'],
        'UNFINISHED': ['UNFINISHED']
    },
    'Cooling':{
        'CENTRAL': ['CENTRAL', 'REFRIGERATION'],
        'WALL': ['WALL']
    },
    'Heating':{
        'FORCED AIR': ['FORCED AIR', 'HEAT PUMP'],
        'RADIANT': ['RADIANT', 'BASEBOARD', 'GAS', 'STEAM']
    },
    'Parking_Features':{
        'ATTACHED GARAGE': ['ATTACHED GARAGE'],
        'DETACHED GARAGE': ['DETACHED GARAGE'],
        'OFF STREET': ['OFF STREET'],
        'ON STREET': ['ON STREET']
    },
    'Architectural_Style':{
        'BUNGALOW': ['BUNGALOW'],
        'COLONIAL': ['COLONIAL'],
        'RANCH': ['RANCH'],
        'APARTMENT': ['APARTMENT', 'COMMON ENTRY'],
        'CAPE COD': ['CAPE COD'],
        'TUDOR': ['TUDOR'],
        'LOFT': ['LOFT'],
        'HISTORIC':['HISTORIC']
    }
    
}

selected_cat_cols = [
    'Parking_Features',
    'Heating',
    'Cooling',
    'Basement',
    'Roof',
    'Structure_Type',
    'Construction_Materials',
    'Exterior_Features',
    'Architectural_Style'
]

## Functions

In [38]:
def get_zipcode(data_column):
    zip_code_list = []
    for index, row in data_column.iteritems():
        #print(row)
        zip_code_list.append(int(re.findall("48\d{3}", row)[0]))
    return pd.DataFrame(data=zip_code_list, index=data_column.index, columns=['Zip_Code'])

def replace_nan_median(data_col, remove_outliers_binary):
    if remove_outliers_binary:
        col_median = data_col[np.abs(data_col-data_col.mean()) <= (3*data_col.std())].median()
        return data_col.fillna(col_median)
    else:
        col_median = data_col.median()
        return data_col.fillna(col_median)

def string_to_categories(data, col_list, mapping_dict):
    df_cat = pd.DataFrame(data=None, index=data.index)
    for col in col_list:
        
        if col == 'Structure_Type':
            temp_col = data[col].str.upper().fillna('NO DATA')
            cat_values = list(temp_col.unique()[pd.notna(temp_col.unique())])
            cat_type = CategoricalDtype(categories=cat_values, ordered=True)
            df_cat[str(col)+'_Cat'] = temp_col.astype(cat_type)
            continue
            
        if col == 'Roof':
            temp_col = data[col].str.upper().replace('COMPOSITION', 'ASPHALT').fillna('NO DATA')
            cat_values = list(temp_col.unique()[pd.notna(temp_col.unique())])
            cat_type = CategoricalDtype(categories=cat_values, ordered=True)
            df_cat[str(col)+'_Cat'] = temp_col.astype(cat_type)
            continue
            
        cat_values = ['NO DATA', 'OTHER'] + list(mapping_dict[col].keys())
        
        if col in ['Parking_Features', 'Cooling', 'Basement']:
            cat_values = cat_values + ['NONE']
        
        temp_dict = {}
        temp_col = data[col].str.replace('\/', ', ', regex=True).str.upper()
        
        for index, row in temp_col.iteritems():
            
            if pd.isna(row) or (row == 'NONE') or (row == 'NO DATA'):
                temp_dict.update({index: row})
                continue
                
            temp_row = row.split(', ')
            temp_row.sort()
            
            for val in temp_row:
                for cat in mapping_dict[col].keys():
                    
                    if val in mapping_dict[col][cat]:
                        temp_dict.update({index: cat})
                        break
                    else:
                        temp_dict.update({index: 'OTHER'})
                break
        
        temp_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=[str(col)+'_Cat']).fillna('NO DATA')
        cat_type = CategoricalDtype(categories=cat_values, ordered=True)
        temp_df = temp_df.astype(cat_type)
        #temp_df.columns = [str(col)+'_Cat']
        df_cat = df_cat.merge(temp_df, left_index=True, right_index=True)
    return df_cat

def load_Dictionary(path, file_name):
    with open(path + str(file_name), "rb") as input_file:
        data_dict = pickle.load(input_file)
    return data_dict
def make_series(name, series_name):
    return pd.Series(data=name, name=series_name)
def force_numeric(column, fill_value):
    if fill_value == None:
        return pd.to_numeric(column, errors='coerce')
    return pd.to_numeric(column, errors='coerce').fillna(fill_value)